In [1]:
import mlrun

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
source = 'git://github.com/GiladShapira94/load-project-example.git#master'

Buy Specify url automatically MLRun clobe your git repository, if Clone = True, always clone (delete any existing content

In [3]:
project = mlrun.load_project('./clone',url=source,init_git=True,clone=True)

In [4]:
print(project.to_yaml())

kind: project
metadata:
  name: from-git
spec:
  functions:
  - name: nuclio
    kind: nuclio
    image: mlrun/mlrun
    handler: multi:multi_3
    with_repo: true
  - name: my_job
    kind: job
    image: mlrun/mlrun
    handler: my_job.func
    with_repo: true
  - url: serving.yaml
    name: serving
    with_repo: true
  workflows: []
  artifacts: []
  source: git://github.com/GiladShapira94/load-project-example.git#refs/heads/master
  origin_url: git://github.com/GiladShapira94/load-project-example.git#refs/heads/master
  load_source_on_run: true
  desired_state: online



In [5]:
project.run_function(function='my_job',params={'num':3},local=True)

> 2022-07-24 14:10:14,079 [warning] run command, file or code were not specified
> 2022-07-24 14:10:14,081 [info] starting run my_job-func uid=8f8bf5bb3f9b4414928bf64e161f49d3 DB=http://mlrun-api:8080
24


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
from-git,...161f49d3,0,Jul 24 14:10:14,completed,my_job-func,v3io_user=giladkind=localowner=giladhost=jupyter-gilad-7779d84ffc-lmkfg,,num=3,num=24return=24,


> 2022-07-24 14:10:14,437 [info] run executed, status=completed


In [6]:
project.run_function(function='my_job',params={'num':3},local=False)

> 2022-07-24 14:10:20,757 [info] starting run my_job-func uid=8b097c2941a640ee8581f1df07934687 DB=http://mlrun-api:8080
> 2022-07-24 14:10:20,988 [info] Job is running in the background, pod: my-job-func-zcthk
> 2022-07-24 14:10:25,158 [info] extracting source from git://github.com/GiladShapira94/load-project-example.git#refs/heads/master to /mlrun/code
24
> 2022-07-24 14:10:25,299 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
from-git,...07934687,0,Jul 24 14:10:25,completed,my_job-func,v3io_user=giladkind=jobowner=giladmlrun/client_version=1.1.0-rc12host=my-job-func-zcthk,,num=3,num=24return=24,


> 2022-07-24 14:10:27,403 [info] run executed, status=completed


In [17]:
serving_func = project.build_function('func')

In [18]:
serving_server = serving_func.to_mock_server()

> 2022-07-24 14:13:51,799 [warning] run command, file or code were not specified


In [19]:
serving_server.test('/',5)

Start
End


13

In [21]:
project.set_source(source,pull_at_runtime=False)
print(project.to_yaml())

kind: project
metadata:
  name: from-git
spec:
  functions:
  - name: nuclio
    kind: nuclio
    image: mlrun/mlrun
    handler: multi:multi_3
    with_repo: true
  - name: my_job
    kind: job
    image: mlrun/mlrun
    handler: my_job.func
    with_repo: true
  - url: serving.yaml
    name: serving
    with_repo: true
  workflows: []
  artifacts: []
  source: git://github.com/GiladShapira94/load-project-example.git#master
  origin_url: git://github.com/GiladShapira94/load-project-example.git#refs/heads/master
  load_source_on_run: false
  desired_state: online



In [22]:
serving_dep = project.deploy_function('serving')

> 2022-07-24 14:14:03,334 [info] Starting remote function deploy
2022-07-24 14:14:18  (info) Deploying function
2022-07-24 14:14:18  (info) Building
2022-07-24 14:14:19  (info) Staging files and preparing base images
2022-07-24 14:14:19  (info) Building processor image
2022-07-24 14:15:34  (info) Build complete
2022-07-24 14:15:42  (info) Function deploy complete
> 2022-07-24 14:15:43,430 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-from-git-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['from-git-serving-from-git.default-tenant.app.cust-cs-il-3-4-1.iguazio-cd1.com/']}


In [23]:
serving_dep.function.invoke('/',5)

> 2022-07-24 14:15:43,499 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-from-git-serving.default-tenant.svc.cluster.local:8080/'}


13

In [26]:
nuclio_func = project.deploy_function('nuclio')

> 2022-07-24 14:18:40,217 [info] Starting remote function deploy
2022-07-24 14:18:55  (info) Deploying function
2022-07-24 14:18:55  (info) Building
2022-07-24 14:18:56  (info) Staging files and preparing base images
2022-07-24 14:18:56  (info) Building processor image
2022-07-24 14:20:31  (info) Build complete
2022-07-24 14:20:37  (info) Function deploy complete
> 2022-07-24 14:20:38,371 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-from-git-nuclio.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['from-git-nuclio-from-git.default-tenant.app.cust-cs-il-3-4-1.iguazio-cd1.com/']}


In [27]:
nuclio_func.function.invoke('/',{'int':4})

> 2022-07-24 14:20:38,445 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-from-git-nuclio.default-tenant.svc.cluster.local:8080/'}


{'int': 12}